#### Data 620 - Project 1 <br> Team 2  <br> June 19, 2019

## <center> Network Analysis </center>

For your first project, you are asked to:
1.  Identify and load a network dataset that has some categorical information available for each node.
2.  For each of the nodes in the dataset, calculate degree centrality and eigenvector centrality.
3.  Compare your centrality measures across your categorical groups.

For example, using the [dataset](http://researchnews.osu.edu/archive/chains.htm) from a study of romantic relationships among high school students in Columbus, Ohio , you might want to determine if there are any differences in your calculated centrality measures between the two sexes.  You might use a t-test or another statistical measure to support your findings.
You may work in a small group.  Your project should be delivered in an IPython Notebook, and posted in GitHub.  Your project is due end of day Monday, June 26th. You and your team should created a video presentation about your project. You should be ready to present your project in our Meet-up on Tuesday, June 27th. 

## Data Source and Description

Link: https://data.cityofnewyork.us/Social-Services/NYC-Women-s-Resource-Network-Database/pqg4-dm6b

The data use we going to work with a dataset from the NYC OpenData and the topic it's about Women Resources Network.
The dataset includes 59 variables by columns and 1,147 rows as recent web update Nov 7, 2018.

We can focus on the categorical variables which can be the organization's name(nodes) and the relations with the services offered and also work with the relation between them by borough.





In [55]:
#importing the libraries 
import networkx as nx
import pandas as pd
import numpy as np

# to import the socrata library you have to go to Anaconda terminal and install the package before importing 
# the library can be install using "pip install sodapy"
from sodapy import Socrata
import warnings 

warnings.filterwarnings('ignore')

In [57]:
# for this dataset we will get the data using Socrata API. client coneection below
client = Socrata("data.cityofnewyork.us",None)

In [37]:
#obtainig the results from the client connection
results = client.get("pqg4-dm6b", limit=2000)

In [38]:
#results length 
len(results)

1147

In [51]:
#putting the the dataset unto a panda Dataframe 
df = pd.DataFrame(results)

In [52]:
#checking the dataframe information 
df.info(verbose= True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1147 entries, 0 to 1146
Data columns (total 57 columns):
address1                                            1074 non-null object
address2                                            145 non-null object
aging                                               1147 non-null object
anti_discrimination_human_rights                    1147 non-null object
arts_culture                                        1147 non-null object
bbl                                                 758 non-null object
bin                                                 758 non-null object
borough                                             771 non-null object
bronx                                               1147 non-null object
brooklyn                                            1147 non-null object
business                                            1147 non-null object
census_tract                                        771 non-null object
child_care_parent_info

In [63]:
#how to subset dataframe
df_new = df[["organizationname","bronx","manhattan","counseling_support_groups","health"]]
df_new

,organizationname,bronx,manhattan,counseling_support_groups,health
0,YWCA of the City of New York,N,Y,Y,N
1,RightRides for Women's Safety,Y,Y,N,N
2,Charles B. Wang Community Health Center,N,Y,Y,Y
3,American-Italian Cancer Foundation's Mobile Ma...,Y,Y,N,Y
4,QHC-Diabetes Center,N,N,N,Y
5,Legal Momentum,Y,Y,N,N
6,Young Mothers Program/BBBS,Y,Y,N,N
7,Shorefront YM-YWHA,N,N,Y,Y
8,Federally Employed Women (FEW),Y,Y,N,N
9,Sauti Yetu Center for African,Y,Y,Y,Y
